## Inference under FHE for the MNIST Dataset using helayers

In this demo, we'll deal with a classification problem for the MNIST dataset [1], trying to correctly classify a batch of samples using a neural network model that will be created and trained using the Keras library (with architecture similar to reference [2]).
First, we'll build a plain neural network for the MNIST model. Then, we'll encrypt the trained network and run inference over it using FHE.

In [ ]:
import os

##### For reproducibility
seed_value= 1
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.random.set_seed(seed_value)
from tensorflow.keras import backend as K

from tensorflow.keras import utils, losses
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
import h5py

# import activations
import sys
sys.path.append(os.path.join('.', 'data_gen'))
from activations import SquareActivation

PATH = os.path.join('data', 'net_mnist')
if not os.path.exists(PATH):
    os.makedirs(PATH)

batch_size = 500
epochs = 10
print("Misc. initializations")

### Load and Preprocess the MNIST Dataset. 

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train /= 255
x_test /= 255
print('data ready')

# Image padding
x_train = np.pad(x_train, ((0, 0), (0, 1), (0, 1), (0, 0)))
x_test = np.pad(x_test, ((0, 0), (0, 1), (0, 1), (0, 0)))
print('Added padding. New shape: ',x_train.shape)

In [ ]:
# Create validation data
testSize=16
x_val = x_test[:-testSize]
x_test = x_test[-testSize:]
y_val = y_test[:-testSize]
y_test = y_test[-testSize:]
print('Validation and test data ready')

# Convert class vector to binary class matrices
num_classes = 10
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)
y_val = utils.to_categorical(y_val, num_classes)

input_shape = x_train[0].shape
print(f'input shape: {input_shape}')

### Save Dataset

In [ ]:
def save_data_set(x, y, data_type, s=''):
    fname=os.path.join(PATH, f'x_{data_type}{s}.h5')
    print("Saving x_{} of shape {} in {}".format(data_type, x.shape,fname))
    xf = h5py.File(fname, 'w')
    xf.create_dataset('x_{}'.format(data_type), data=x)
    xf.close()

    yf = h5py.File(os.path.join(PATH, f'y_{data_type}{s}.h5'), 'w')
    yf.create_dataset(f'y_{data_type}', data=y)
    yf.close()

save_data_set(x_test, y_test, data_type='test')
save_data_set(x_train, y_train, data_type='train')
save_data_set(x_val, y_val, data_type='val')

### Build a Plain Neural Network for the MNIST Model

In [ ]:
model = Sequential()
model.add(Conv2D(filters=5, kernel_size=5, strides=2, padding='valid',
                 input_shape=input_shape))
model.add(Flatten())
model.add(SquareActivation())
model.add(Dense(100))
model.add(SquareActivation())
model.add(Dense(num_classes))

model.summary()

### Train the Neural Network

In [ ]:
def sum_squared_error(y_true, y_pred):
    return K.sum(K.square(y_pred - y_true), axis=-1)

model.compile(loss=sum_squared_error,
                  optimizer='Adam',
                  metrics=['accuracy'])

model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=2,
              validation_data=(x_val, y_val),
              shuffle=True,
              )

score = model.evaluate(x_test, y_test, verbose=0)

print(f'Test loss: { score[0]:.3f}')
print(f'Test accuracy: {score[1] * 100:.3f}%')

### Report the Confusion Matrix of the Trained Model

In [ ]:
from sklearn import metrics

y_pred_vals = model.predict(x_test)
y_pred = np.argmax(y_pred_vals, axis=1)
y_test = np.argmax(y_test, axis=1)
cm = metrics.confusion_matrix(y_test, y_pred)
print(cm)

### Serialize Model and Weights

In [ ]:
model_json = model.to_json()
with open(os.path.join(PATH, 'model.json'), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(PATH, 'model.h5'))
print("Saved model to disk")

We are all done training the plain network. Next we will encrypt the network and run inference over it using FHE. Let's start with some initializations.

In [ ]:
import pyhelayers
import utils

utils.verify_memory()

print('Misc. initalizations')

Here's a general outline of what we're about to do:

First, we'll build a plain neural network model, loading it from files that were created and saved above.
Then, an automated optimizer will examine our network and will provide us with an HE profile: various configuration details that will allow running inference under encryption efficiently.
Finally, we'll construct a neural net object: an encrypted version of our network. We'll demonstrate how we can encrypt data, run inference on our encrypted network, and compare the results against the expected labels.
Now let's dive in . . .

First, we define a variable "context" to represent the library. Note that it is still empty and will only be initialized later, once we know what configuration should we use depending on the NN model we'll build and on our requirements of the inference process

In [ ]:
context = pyhelayers.DefaultContext()
print('HE context ready')

We would like to create our NN model encrypted under HE. We'll first build a plain NN model and then encrypt it to receive an encrypted version of the model. We'll declare a variable "nnp" to represent the plain NN.

In [ ]:
nnp = pyhelayers.NeuralNetPlain()

Next, we initialize the NN architecture and weights. The architecture describes the different layers that form the NN and information on each. We'll load it from the JSON file that was created while building and training the model. We demonstrate how to load the weights from a pre-defined HDF5 file created in Python, containing the weights in the plain.

In [ ]:
hyper_params = pyhelayers.PlainModelHyperParams()
nnp.init_from_files(hyper_params, [os.path.join(PATH, "model.json"), os.path.join(PATH, "model.h5")])
print("loaded plain model")

And now for the interesting part. We'll build an encrypted version of our NN model. But in order to do that, we first need to define a configuration of the library and of the layout of each ciphertext, which we call a "tile".  
Defining all the right parameters for the library and for the layout is some hard work. It is complex and subtle, since the configuration includes multiple HE-related parameters, many of them depend on one another and on the specific NN model we defined, and the values we choose will have great influence on the security, accuracy and performance of the inference process.  
In order to avoid this complexity, we'll use the HE profile optimizer. This optimizer helps us find a configuration that's guaranteed to be secure and feasible. Furthermore, the optimizer receives the plain NN we've built and considers what's optimal for this very model in terms of performance.  
We can notify the optimizer of various requirements we have for running the model, with respect to the library and packaging considerations. For example here we ask to optimize for a batch size of 16 samples.
The optimizer is called when compiling the model, given HE run requirements and the plain model.
The result returned is a model "profile", describing how the library and the encrypted NN should be initialized.

In [ ]:
he_run_req = pyhelayers.HeRunRequirements()
he_run_req.set_he_context_options([pyhelayers.DefaultContext()])
he_run_req.optimize_for_batch_size(16)

profile = pyhelayers.HeModel.compile(nnp, he_run_req)
batch_size = profile.get_optimal_batch_size()
print('Profile ready. Batch size=',batch_size)

The HE profile includes a set of requirements from the library, such as the security level, precision, size of ciphertext, multiplication depth etc. Some of these can be set by the user and some others were found by the optimizer based on the NN architecture we defined. We'll now take this requirement object and use it to initialize the library.

In [ ]:
context = pyhelayers.HeModel.create_context(profile)
print('HE context initalized')

We will now build our encrypted NN. We start by defining an empty HE NN, providing it with the library.

Now we initialize the NN and populate its weights. Since we have already built a plain NN describing the model architecture and containing the weights, we'll simply encrypt into an encrypted version of the NN. Notice we provide the profile that was recommended by the optimizer.

In [ ]:
nn = pyhelayers.NeuralNet(context)
nn.encode_encrypt(nnp, profile)
print('Encrypted network ready')

We will now load real samples of the MNIST dataset to classify. We will load the samples and the corresponding true labels from HDF5 files. We will also extract the first batch of samples and labels.

In [ ]:
with h5py.File(os.path.join(PATH, "x_test.h5")) as f:
    x_test = np.array(f["x_test"])
with h5py.File(os.path.join(PATH, "y_test.h5")) as f:
    y_test = np.array(f["y_test"])
    
plain_samples, labels = utils.extract_batch(x_test, y_test, batch_size, 0)

print('Batch of size',batch_size,'loaded')

To populate the input, we need to encode and then encrypt the values of the plain input under HE.

In [ ]:
iop = nn.create_io_processor()
samples = pyhelayers.EncryptedData(context)
iop.encode_encrypt_inputs_for_predict(samples, [plain_samples])
print('Test data encrypted')

We now go ahead with the inference itself. We run the encrypted input through the encrypted NN to obtain encrypted results. This computation does not use the secret key and acts on completely encrypted values. Running the inference is done using the "predict" method of the NN, that receives the destination 3D structure to put the result of the computation in, and the input for the inference.

In [ ]:
utils.start_timer()

predictions = pyhelayers.EncryptedData(context)
nn.predict(predictions, samples)

duration=utils.end_timer('predict')
utils.report_duration('predict per sample',duration/batch_size)

In order to assess the results of the computation, we first need to decrypt them. This is done by an IO processor that has the secret key and is capable of decrypting the ciphertext and decoding it into plaintext version of the HE computation result.

In [ ]:
plain_predictions = iop.decrypt_decode_output(predictions)
print('predictions',plain_predictions)

Now we compare the results against the expected labels and compute the confusion matrix and the accuracy.

In [ ]:
utils.assess_results(labels, plain_predictions)

<br>

References:

<sub><sup> 1.	LeCun, Yann and Cortes, Corinna. "MNIST handwritten digit database." (2010): </sup></sub>

<sub><sup> 2.	Gilad-Bachrach, R., Dowlin, N., Laine, K., Lauter, K., Naehrig, M. &amp; Wernsing, J.. (2016). CryptoNets: Applying Neural Networks to Encrypted Data with High Throughput and Accuracy. Proceedings of The 33rd International Conference on Machine Learning, in Proceedings of Machine Learning Research 48:201-210 Available from https://proceedings.mlr.press/v48/gilad-bachrach16.html.
</sup></sub>
